# Overview

This scraper loads the TMDB data file with IMDB IDs, containing 10,000 movie entries, and scrapes the Box Office Mojo website for any/all of the following:
    * budget
    * domestic gross
    * worldwide gross
    * studio
    * MPAA rating

# Library Imports

In [1]:
import pandas as pd
import requests
import re
import numpy as np
from bs4 import BeautifulSoup as bs
import time
from random import randint
from time import sleep

# Scraping Functions

In [2]:
# function that converts our string to a search string
def format_title(title):
    text = str(title)
    result = re.sub(r"[,@\'?\.$%_:â()-]", "", text, flags=re.I)
    result = re.sub(r"\s+","+", result, flags = re.I)
    return result

In [3]:
# function to convert the raw numbers into integers
def convert_number(x):
    '''Takes in a string formatted number that starts with $ and may include commas, and returns that 
    number as a whole integer that can be used in calculations'''
    try:
        x = x[1:]
        x = x.replace(',', '')
        x = int(x)
        return x
    except:
        print('No number to convert')
        return None

In [4]:
# function that receives the followurl appendation for the IMDB id and follows it to Box Office Mojo
def follow_bom(movie_id):
    followurl = 'https://www.boxofficemojo.com/title/' + movie_id + '/?ref_=bo_hm_rs'
    page = requests.get(followurl)
    soup = bs(page.content, 'html.parser')
    return soup

In [5]:
# function that receives the followurl appendation for the IMDB id and follows it to Box Office Mojo
def follow_imdb(movie_id):
    followurl = 'https://www.imdb.com/title/' + movie_id + '/?ref_=tt_sims_tt'
    page = requests.get(followurl)
    soup = bs(page.content, 'html.parser')
    return soup

In [6]:
# function that takes in the product page and gets the domestic and worldwide gross
def gross(movie_page):
    try:
        search1 = movie_page.find('div', class_='a-section a-spacing-none mojo-performance-summary-table')
        div_search = search1.find_all('div', class_='a-section a-spacing-none')
        
        try:
            dgross = div_search[0].find_all("span", class_='money')[0].text
            dgross = convert_number(dgross)
        except:
            dgross = None
        
        try:
            wgross = div_search[2].find_all("span", class_='money')[0].text
            wgross = convert_number(wgross)
        except:
            wgross = None
            
        return dgross, wgross
    
    except:
        return None, None

In [7]:
# function that takes in the product page and gets the studio name
def studio_name(soup):
    try:
        search1 = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        search2 = search1.find_all('div', class_='a-section a-spacing-none')[0].find_all("span")
        if search2[0].text == 'Domestic Distributor':
            studio = search2[1].text.replace('See full company information', '').rstrip()
            return studio
        else:
            return None
    except:
        print('error in studio function')
        return None

In [8]:
# function that takes in the product page and gets the budget
def find_budget(soup):
    try:
        search1 = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        search2 = search1.find_all('div', class_='a-section a-spacing-none')[2].find_all("span")
        if search2[0].text == 'Budget':
            budget = search2[1].text
            budget = convert_number(budget)
            return budget
        else:
            return None
    except:
        print('error in budget function')
        return None

In [9]:
# function that takes in the product page and gives the MPAA rating
def find_mpaa(soup):
    try:
        search1 = soup.find('div', class_='titleBar')
        search2 = search1.find('div', class_='subtext')
        rating = search2.text.lstrip()
        rating = rating[:5].rstrip()
        if rating == 'G' or rating == 'PG' or rating == 'PG-13' or rating == 'R' or rating == 'NC-17' or rating == 'NR':
            return rating
        else:
            return 0
        print(rating)
        #return rating
    except:
        print('error in rating function')
        return None

In [20]:
# Master Scraping Function

def webscraper(scrapingset):
    count = 1
    length = scrapingset.shape[0]
    ts = time.time()
    for ind in scrapingset.index:   
        movie_id = str(scrapingset['id'][ind])
        print(('Item {} / {} - {}').format(count, length, movie_id))
        movie_page = follow_bom(movie_id)
        count += 1

        studio = studio_name(movie_page)
        scrapingset['studio'][ind] = studio
   
        dgross, wgross = gross(movie_page)
        scrapingset['dom_gross'][ind] = dgross
        scrapingset['world_gross'][ind] = wgross
    
        budget = find_budget(movie_page)
        scrapingset['budget'][ind] = budget
        
        movie_page2 = follow_imdb(movie_id)
        rating = find_mpaa(movie_page2)
        scrapingset['rating'][ind] = rating
        
        sleep(randint(0,2))
    
    tnow = time.time()
    duration = round((tnow - ts), 2)
    scrape_average = round(duration/length, 2)
    print('{} minutes elapsed'.format(duration/60))
    print('{} seconds per item'.format(scrape_average))
    return scrapingset

# TO DO BEFORE REDOING

Round the numbers so they don't have decimal places

Add a scraper for rating!

# Data Import

In [11]:
# import tmdb_imdb files
df = pd.read_csv('api_data/tmdb_imdb_ids.csv')

In [12]:
df

,id,popularity,vote_count,genre_ids,title,vote_average,release_date
0,tt2380307,520.621,12639,"[16, 10751, 35, 12, 14, 10402]",Coco,8.2,2017-10-27
1,tt7286456,330.357,15378,"[80, 53, 18]",Joker,8.2,2019-10-02
2,tt7349950,288.149,5133,"[27, 14]",It Chapter Two,6.9,2019-09-04
3,tt4520988,257.243,6344,"[16, 10751, 12, 35, 14]",Frozen II,7.3,2019-11-20
4,tt7975244,216.184,4925,"[12, 35, 14]",Jumanji The Next Level,7.0,2019-12-04
...,...,...,...,...,...,...,...
9995,tt0349080,4.837,61,"[35, 18, 10749]",All About Anna,3.2,2005-11-24
9996,tt0366178,4.836,67,"[16, 878]",Program,6.9,2003-02-07
9997,tt2501618,4.830,72,"[27, 14, 16]",Dont Hug Me Im Scared,7.5,2011-07-25
9998,tt0340376,4.830,40,"[18, 53]",The Statement,5.7,2003-12-12


In [13]:
#We need nothing but the id from this dataframe
df.drop(columns=['popularity', 'vote_count', 'genre_ids', 'title', 'vote_average', 'release_date'], inplace=True)

In [14]:
# We're prepping our dataframe structure for the information that we plan to scrape
df['studio'] = ''
df['rating'] = ''
df['budget'] = ''
df['dom_gross'] = ''
df['world_gross'] = ''
df

,id,studio,rating,budget,dom_gross,world_gross
0,tt2380307,,,,,
1,tt7286456,,,,,
2,tt7349950,,,,,
3,tt4520988,,,,,
4,tt7975244,,,,,
...,...,...,...,...,...,...
9995,tt0349080,,,,,
9996,tt0366178,,,,,
9997,tt2501618,,,,,
9998,tt0340376,,,,,


# Testing Set

Before we scrape for 10k returns, we will do a small test scrape

In [15]:
test1 = pd.DataFrame(columns = ['id', 'rating', 'studio', 'budget', 'dom_gross', 'world_gross'])

In [16]:
ids = ['tt1825683', 'tt0349080', 'tt7286456', 'tt3120408']
test1['id'] = np.array(ids)

In [17]:
test1

,id,rating,studio,budget,dom_gross,world_gross
0,tt1825683,NaN,NaN,NaN,NaN,NaN
1,tt0349080,NaN,NaN,NaN,NaN,NaN
2,tt7286456,NaN,NaN,NaN,NaN,NaN
3,tt3120408,NaN,NaN,NaN,NaN,NaN


In [18]:
test1 = webscraper(test1)

Item 1 / 4 - tt1825683
Item 2 / 4 - tt0349080
Item 3 / 4 - tt7286456
Item 4 / 4 - tt3120408
0.28833333333333333 minutes elapsed
4.33 seconds per item


In [19]:
test1

,id,rating,studio,budget,dom_gross,world_gross
0,tt1825683,PG-13,Walt Disney Studios Motion Pictures,None,700426566,1347462384
1,tt0349080,0,None,None,None,None
2,tt7286456,R,Warner Bros.,55000000,335451311,1074251311
3,tt3120408,G,None,None,None,31836701


In [21]:
#A bigger test set
test2 = df[0:10]
test2

,id,studio,rating,budget,dom_gross,world_gross
0,tt2380307,,,,,
1,tt7286456,,,,,
2,tt7349950,,,,,
3,tt4520988,,,,,
4,tt7975244,,,,,
5,tt4154756,,,,,
6,tt5168832,,,,,
7,tt9465600,,,,,
8,tt0414982,,,,,
9,tt6806448,,,,,


In [22]:
test2 = webscraper(test2)

Item 1 / 10 - tt2380307


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 10 - tt7286456
Item 3 / 10 - tt7349950
Item 4 / 10 - tt4520988
Item 5 / 10 - tt7975244
Item 6 / 10 - tt4154756
Item 7 / 10 - tt5168832
Item 8 / 10 - tt9465600
Item 9 / 10 - tt0414982
Item 10 / 10 - tt6806448
0.4658333333333333 minutes elapsed
2.79 seconds per item


In [23]:
test2

,id,studio,rating,budget,dom_gross,world_gross
0,tt2380307,Walt Disney Studios Motion Pictures,PG,None,210460015,807817888
1,tt7286456,Warner Bros.,R,55000000,335451311,1074251311
2,tt7349950,Warner Bros.,R,79000000,211593228,473093228
3,tt4520988,Walt Disney Studios Motion Pictures,PG,150000000,477373578,1450026933
4,tt7975244,Sony Pictures Entertainment (SPE),PG-13,125000000,320314960,800059707
5,tt4154756,Walt Disney Studios Motion Pictures,PG-13,None,678815482,2048359754
6,tt5168832,None,0,None,None,None
7,tt9465600,None,R,None,None,None
8,tt0414982,New Line Cinema,R,25000000,54098051,118890272
9,tt6806448,Universal Pictures,PG-13,200000000,173956935,759056935


In [ ]:
^break

# The Big Scrape

We're ready to do the big scrape!
We'll break our frame of 10,000 entries into 10 smaller ones in case of any errors.

In [2]:
# import partially finished scrape file if needed
df = pd.read_csv('api_data/tmdb_bom_scraped.csv')

In [24]:
df1 = df[0:1000]
df2 = df[1000:2000]
df3 = df[2000:3000]
df4 = df[3000:4000]
df5 = df[4000:5000]
df6 = df[5000:6000]
df7 = df[6000:7000]
df8 = df[7000:8000]
df9 = df[8000:9000]
df10 = df[9000:10000]

In [25]:
df1 = webscraper(df1)

Item 1 / 1000 - tt2380307


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt7286456
Item 3 / 1000 - tt7349950
Item 4 / 1000 - tt4520988
Item 5 / 1000 - tt7975244
Item 6 / 1000 - tt4154756
Item 7 / 1000 - tt5168832
Item 8 / 1000 - tt9465600
Item 9 / 1000 - tt0414982
Item 10 / 1000 - tt6806448
Item 11 / 1000 - tt4126476
Item 12 / 1000 - tt6450804
Item 13 / 1000 - tt6105098
Item 14 / 1000 - tt4154796
Item 15 / 1000 - tt4777008
Item 16 / 1000 - tt4094724
Item 17 / 1000 - tt6133466
Item 18 / 1000 - tt8579674
Item 19 / 1000 - tt6320628
Item 20 / 1000 - tt1935859
Item 21 / 1000 - tt1979376
Item 22 / 1000 - tt1935896
Item 23 / 1000 - tt6146586
Item 24 / 1000 - tt2527338
Item 25 / 1000 - tt4281724
Item 26 / 1000 - tt1025100
Item 27 / 1000 - tt2709692
Item 28 / 1000 - tt4633694
Item 29 / 1000 - tt0329774
Item 30 / 1000 - tt1560220
Item 31 / 1000 - tt8106534
Item 32 / 1000 - tt7287680
Item 33 / 1000 - tt2184339
Item 34 / 1000 - tt1620981
Item 35 / 1000 - tt6095472
Item 36 / 1000 - tt5884052
Item 37 / 1000 - tt5220122
Item 38 / 1000 - tt6139732
Item 39 /

Item 299 / 1000 - tt6643972
Item 300 / 1000 - tt9382596
Item 301 / 1000 - tt7831358
Item 302 / 1000 - tt8186318
Item 303 / 1000 - tt4007502
Item 304 / 1000 - tt1790809
Item 305 / 1000 - tt6645614
Item 306 / 1000 - tt1267297
Item 307 / 1000 - tt0434409
Item 308 / 1000 - tt8752440
Item 309 / 1000 - tt1979388
Item 310 / 1000 - tt3584496
Item 311 / 1000 - tt2582802
Item 312 / 1000 - tt4790268
Item 313 / 1000 - tt0385887
Item 314 / 1000 - tt1413492
Item 315 / 1000 - tt4158096
Item 316 / 1000 - tt0816711
Item 317 / 1000 - tt1411697
Item 318 / 1000 - tt1029360
Item 319 / 1000 - tt1261978
Item 320 / 1000 - tt2911666
Item 321 / 1000 - tt0372784
Item 322 / 1000 - tt0167261
Item 323 / 1000 - tt10767720
Item 324 / 1000 - tt0936501
Item 325 / 1000 - tt3235888
Item 326 / 1000 - tt3846674
Item 327 / 1000 - tt1981115
Item 328 / 1000 - tt1951261
Item 329 / 1000 - tt0252866
Item 330 / 1000 - tt6182908
Item 331 / 1000 - tt0472043
Item 332 / 1000 - tt0449088
Item 333 / 1000 - tt3605418
Item 334 / 1000 - t

Item 592 / 1000 - tt0328828
Item 593 / 1000 - tt1563742
Item 594 / 1000 - tt0373883
Item 595 / 1000 - tt0328880
Item 596 / 1000 - tt1638002
Item 597 / 1000 - tt7339792
Item 598 / 1000 - tt2537176
Item 599 / 1000 - tt8031422
Item 600 / 1000 - tt0780521
Item 601 / 1000 - tt9071322
Item 602 / 1000 - tt4257926
Item 603 / 1000 - tt4950110
Item 604 / 1000 - tt1598778
Item 605 / 1000 - tt0489099
Item 606 / 1000 - tt6292852
Item 607 / 1000 - tt1853728
Item 608 / 1000 - tt5822648
Item 609 / 1000 - tt1179056
Item 610 / 1000 - tt4799066
Item 611 / 1000 - tt0498353
Item 612 / 1000 - tt9815714
Item 613 / 1000 - tt1618434
Item 614 / 1000 - tt1190080
Item 615 / 1000 - tt8235660
Item 616 / 1000 - tt2091256
Item 617 / 1000 - tt0266697
Item 618 / 1000 - tt0134084
Item 619 / 1000 - tt3440298
Item 620 / 1000 - tt6017942
Item 621 / 1000 - tt8179388
Item 622 / 1000 - tt0289043
Item 623 / 1000 - tt3774802
Item 624 / 1000 - tt0317705
Item 625 / 1000 - tt3195644
Item 626 / 1000 - tt2231253
Item 627 / 1000 - tt

Item 883 / 1000 - tt7401588
Item 884 / 1000 - tt2404435
Item 885 / 1000 - tt0451079
Item 886 / 1000 - tt1351685
Item 887 / 1000 - tt7052244
Item 888 / 1000 - tt1139668
Item 889 / 1000 - tt1700841
Item 890 / 1000 - tt5715410
Item 891 / 1000 - tt5360996
Item 892 / 1000 - tt0309593
Item 893 / 1000 - tt1485796
Item 894 / 1000 - tt6857112
Item 895 / 1000 - tt0884328
Item 896 / 1000 - tt0383206
Item 897 / 1000 - tt5737536
Item 898 / 1000 - tt1985966
Item 899 / 1000 - tt5208950
Item 900 / 1000 - tt4824308
Item 901 / 1000 - tt0265208
Item 902 / 1000 - tt0301470
Item 903 / 1000 - tt1217213
Item 904 / 1000 - tt1478839
Item 905 / 1000 - tt0265086
Item 906 / 1000 - tt0840361
Item 907 / 1000 - tt2096672
Item 908 / 1000 - tt4287320
Item 909 / 1000 - tt4976192
Item 910 / 1000 - tt1821694
Item 911 / 1000 - tt0389790
Item 912 / 1000 - tt1192628
Item 913 / 1000 - tt1583421
Item 914 / 1000 - tt6017756
Item 915 / 1000 - tt7040874
Item 916 / 1000 - tt3079380
Item 917 / 1000 - tt1285016
Item 918 / 1000 - tt

In [30]:
df2 = webscraper(df2)

Item 1 / 1000 - tt0780653


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt2515086
Item 3 / 1000 - tt0293662
Item 4 / 1000 - tt2230358
Item 5 / 1000 - tt0988045
Item 6 / 1000 - tt1850397
Item 7 / 1000 - tt4667854
Item 8 / 1000 - tt0287978
Item 9 / 1000 - tt2011159
Item 10 / 1000 - tt0120917
Item 11 / 1000 - tt0209163
Item 12 / 1000 - tt0220506
Item 13 / 1000 - tt0462590
Item 14 / 1000 - tt0377092
Item 15 / 1000 - tt5610626
Item 16 / 1000 - tt2531344
Item 17 / 1000 - tt7634968
Item 18 / 1000 - tt0349903
Item 19 / 1000 - tt4218572
Item 20 / 1000 - tt0409459
Item 21 / 1000 - tt1408101
Item 22 / 1000 - tt1234721
Item 23 / 1000 - tt0130623
Item 24 / 1000 - tt1139592
Item 25 / 1000 - tt2720826
Item 26 / 1000 - tt0362120
Item 27 / 1000 - tt2339741
Item 28 / 1000 - tt9419834
Item 29 / 1000 - tt0302640
Item 30 / 1000 - tt6739094
Item 31 / 1000 - tt1262416
Item 32 / 1000 - tt0430304
Item 33 / 1000 - tt1502712
Item 34 / 1000 - tt4438848
Item 35 / 1000 - tt1146431
Item 36 / 1000 - tt4916630
Item 37 / 1000 - tt5042436
Item 38 / 1000 - tt3532216
Item 39 /

Item 299 / 1000 - tt0440963
Item 300 / 1000 - tt0332379
Item 301 / 1000 - tt3247714
Item 302 / 1000 - tt2719848
Item 303 / 1000 - tt0949731
Item 304 / 1000 - tt0358082
Item 305 / 1000 - tt5437928
Item 306 / 1000 - tt1411250
Item 307 / 1000 - tt0844894
Item 308 / 1000 - tt2170439
Item 309 / 1000 - tt1386703
Item 310 / 1000 - tt0346631
Item 311 / 1000 - tt0454921
Item 312 / 1000 - tt0467421
Item 313 / 1000 - tt5439796
Item 314 / 1000 - tt1426329
Item 315 / 1000 - tt0167190
Item 316 / 1000 - tt4103724
Item 317 / 1000 - tt5692390
Item 318 / 1000 - tt7578566
Item 319 / 1000 - tt6170506
Item 320 / 1000 - tt0429493
Item 321 / 1000 - tt3224458
Item 322 / 1000 - tt4915672
Item 323 / 1000 - tt4218696
Item 324 / 1000 - tt0482571
Item 325 / 1000 - tt0368933
Item 326 / 1000 - tt0356910
Item 327 / 1000 - tt1464540
Item 328 / 1000 - tt0790724
Item 329 / 1000 - tt0870984
Item 330 / 1000 - tt0229260
Item 331 / 1000 - tt5362988
Item 332 / 1000 - tt1197624
Item 333 / 1000 - tt0396171
Item 334 / 1000 - tt

Item 592 / 1000 - tt6450752
Item 593 / 1000 - tt1179933
Item 594 / 1000 - tt0816671
Item 595 / 1000 - tt1705952
Item 596 / 1000 - tt2620590
Item 597 / 1000 - tt1150934
Item 598 / 1000 - tt6499752
Item 599 / 1000 - tt0327554
Item 600 / 1000 - tt1292566
Item 601 / 1000 - tt2387433
Item 602 / 1000 - tt2238032
Item 603 / 1000 - tt1929263
Item 604 / 1000 - tt6911608
Item 605 / 1000 - tt6349302
Item 606 / 1000 - tt0403703
Item 607 / 1000 - tt0366548
Item 608 / 1000 - tt1045658
Item 609 / 1000 - tt1010048
Item 610 / 1000 - tt3152624
Item 611 / 1000 - tt0780504
Item 612 / 1000 - tt3793788
Item 613 / 1000 - tt0475944
Item 614 / 1000 - tt0406759
Item 615 / 1000 - tt2205697
Item 616 / 1000 - tt0396752
Item 617 / 1000 - tt4547056
Item 618 / 1000 - tt5177088
Item 619 / 1000 - tt0304711
Item 620 / 1000 - tt1691917
Item 621 / 1000 - tt1656190
Item 622 / 1000 - tt6791096
Item 623 / 1000 - tt5580036
Item 624 / 1000 - tt1134854
Item 625 / 1000 - tt0960731
Item 626 / 1000 - tt2085930
Item 627 / 1000 - tt

Item 885 / 1000 - tt5794440
Item 886 / 1000 - tt1800246
Item 887 / 1000 - tt0290002
Item 888 / 1000 - tt4587656
Item 889 / 1000 - tt1454029
Item 890 / 1000 - tt0898367
Item 891 / 1000 - tt4209788
Item 892 / 1000 - tt0454945
Item 893 / 1000 - tt5612742
Item 894 / 1000 - tt1336608
Item 895 / 1000 - tt2139555
Item 896 / 1000 - tt3401882
Item 897 / 1000 - tt0437179
Item 898 / 1000 - tt5314190
Item 899 / 1000 - tt0424228
Item 900 / 1000 - tt1666801
Item 901 / 1000 - tt1753968
Item 902 / 1000 - tt1950235
Item 903 / 1000 - tt1033643
Item 904 / 1000 - tt1294226
Item 905 / 1000 - tt0452702
Item 906 / 1000 - tt2290147
Item 907 / 1000 - tt1734493
Item 908 / 1000 - tt9028856
Item 909 / 1000 - tt0892318
Item 910 / 1000 - tt0800039
Item 911 / 1000 - tt0375173
Item 912 / 1000 - tt1142988
Item 913 / 1000 - tt0451957
Item 914 / 1000 - tt0386588
Item 915 / 1000 - tt1272878
Item 916 / 1000 - tt0200465
Item 917 / 1000 - tt0329691
Item 918 / 1000 - tt1037705
Item 919 / 1000 - tt0427392
Item 920 / 1000 - tt

In [34]:
df3 = webscraper(df3)

Item 1 / 1000 - tt4581576


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt0481369
Item 3 / 1000 - tt3787590
Item 4 / 1000 - tt0274546
Item 5 / 1000 - tt5093026
Item 6 / 1000 - tt0288045
Item 7 / 1000 - tt1430612
Item 8 / 1000 - tt1814614
Item 9 / 1000 - tt0443473
Item 10 / 1000 - tt0407121
Item 11 / 1000 - tt6217804
Item 12 / 1000 - tt1596365
Item 13 / 1000 - tt2476918
Item 14 / 1000 - tt7549996
Item 15 / 1000 - tt1579247
Item 16 / 1000 - tt0960890
Item 17 / 1000 - tt0381849
Item 18 / 1000 - tt3451230
Item 19 / 1000 - tt2180411
Item 20 / 1000 - tt7319822
Item 21 / 1000 - tt9381998
Item 22 / 1000 - tt3467412
Item 23 / 1000 - tt6946580
Item 24 / 1000 - tt0970452
Item 25 / 1000 - tt0443701
Item 26 / 1000 - tt0485947
Item 27 / 1000 - tt4677938
Item 28 / 1000 - tt4943998
Item 29 / 1000 - tt7983844
Item 30 / 1000 - tt6048930
Item 31 / 1000 - tt1564585
Item 32 / 1000 - tt8457394
Item 33 / 1000 - tt2752758
Item 34 / 1000 - tt1560985
Item 35 / 1000 - tt0303933
Item 36 / 1000 - tt0806203
Item 37 / 1000 - tt1205535
Item 38 / 1000 - tt1288403
Item 39 /

Item 298 / 1000 - tt4807408
Item 299 / 1000 - tt2100546
Item 300 / 1000 - tt0964517
Item 301 / 1000 - tt0442933
Item 302 / 1000 - tt1599351
Item 303 / 1000 - tt1002563
Item 304 / 1000 - tt2172934
Item 305 / 1000 - tt0491203
Item 306 / 1000 - tt1447972
Item 307 / 1000 - tt2205401
Item 308 / 1000 - tt0283426
Item 309 / 1000 - tt2039338
Item 310 / 1000 - tt5698320
Item 311 / 1000 - tt0334541
Item 312 / 1000 - tt3164256
Item 313 / 1000 - tt0317198
Item 314 / 1000 - tt1596363
Item 315 / 1000 - tt0206275
Item 316 / 1000 - tt0457419
Item 317 / 1000 - tt2349460
Item 318 / 1000 - tt0280590
Item 319 / 1000 - tt8722440
Item 320 / 1000 - tt7985692
Item 321 / 1000 - tt6182078
Item 322 / 1000 - tt1925431
Item 323 / 1000 - tt0405469
Item 324 / 1000 - tt1971325
Item 325 / 1000 - tt6019206
Item 326 / 1000 - tt5502946
Item 327 / 1000 - tt1205489
Item 328 / 1000 - tt5513260
Item 329 / 1000 - tt1477715
Item 330 / 1000 - tt8510488
Item 331 / 1000 - tt1535109
Item 332 / 1000 - tt1251757
Item 333 / 1000 - tt

Item 590 / 1000 - tt0202470
Item 591 / 1000 - tt0469111
Item 592 / 1000 - tt0758758
Item 593 / 1000 - tt5563330
Item 594 / 1000 - tt1179034
Item 595 / 1000 - tt2870756
Item 596 / 1000 - tt2582784
Item 597 / 1000 - tt2513074
Item 598 / 1000 - tt0865957
Item 599 / 1000 - tt0844760
Item 600 / 1000 - tt3598222
Item 601 / 1000 - tt0454848
Item 602 / 1000 - tt0486761
Item 603 / 1000 - tt2315596
Item 604 / 1000 - tt3289724
Item 605 / 1000 - tt1196339
Item 606 / 1000 - tt1212419
Item 607 / 1000 - tt4682786
Item 608 / 1000 - tt4215766
Item 609 / 1000 - tt1259998
Item 610 / 1000 - tt0976051
Item 611 / 1000 - tt0245562
Item 612 / 1000 - tt0457275
Item 613 / 1000 - tt5044656
Item 614 / 1000 - tt0221027
Item 615 / 1000 - tt6997574
Item 616 / 1000 - tt4797160
Item 617 / 1000 - tt0268397
Item 618 / 1000 - tt0889583
Item 619 / 1000 - tt8485364
Item 620 / 1000 - tt1024255
Item 621 / 1000 - tt1131724
Item 622 / 1000 - tt5390066
Item 623 / 1000 - tt2396421
Item 624 / 1000 - tt0866439
Item 625 / 1000 - tt

Item 883 / 1000 - tt4652532
Item 884 / 1000 - tt0360139
Item 885 / 1000 - tt0465624
Item 886 / 1000 - tt0239641
Item 887 / 1000 - tt0120913
Item 888 / 1000 - tt0815236
Item 889 / 1000 - tt0326769
Item 890 / 1000 - tt4009278
Item 891 / 1000 - tt2172935
Item 892 / 1000 - tt7358936
Item 893 / 1000 - tt0274166
Item 894 / 1000 - tt2282016
Item 895 / 1000 - tt0238546
Item 896 / 1000 - tt5851786
Item 897 / 1000 - tt3352390
Item 898 / 1000 - tt0881320
Item 899 / 1000 - tt1186373
Item 900 / 1000 - tt0278488
Item 901 / 1000 - tt2279339
Item 902 / 1000 - tt7689966
Item 903 / 1000 - tt2582502
Item 904 / 1000 - tt0190590
Item 905 / 1000 - tt0183649
Item 906 / 1000 - tt0413895
Item 907 / 1000 - tt0995868
Item 908 / 1000 - tt0263757
Item 909 / 1000 - tt1600058
Item 910 / 1000 - tt1282052
Item 911 / 1000 - tt0356618
Item 912 / 1000 - tt1135084
Item 913 / 1000 - tt1120985
Item 914 / 1000 - tt1886644
Item 915 / 1000 - tt2400685
Item 916 / 1000 - tt0387808
Item 917 / 1000 - tt3733774
Item 918 / 1000 - tt

In [36]:
df4 = webscraper(df4)

Item 1 / 1000 - tt0421994


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt1321511
Item 3 / 1000 - tt0947810
Item 4 / 1000 - tt1773753
Item 5 / 1000 - tt1605783
Item 6 / 1000 - tt0884726
Item 7 / 1000 - tt4419364
Item 8 / 1000 - tt2008633
Item 9 / 1000 - tt0443543
Item 10 / 1000 - tt1915581
Item 11 / 1000 - tt0305711
Item 12 / 1000 - tt5688868
Item 13 / 1000 - tt2515030
Item 14 / 1000 - tt0335266
Item 15 / 1000 - tt1754799
Item 16 / 1000 - tt1117385
Item 17 / 1000 - tt0259446
Item 18 / 1000 - tt2702724
Item 19 / 1000 - tt0255798
Item 20 / 1000 - tt1564349
Item 21 / 1000 - tt3531202
Item 22 / 1000 - tt0256415
Item 23 / 1000 - tt0816520
Item 24 / 1000 - tt0377713
Item 25 / 1000 - tt1216492
Item 26 / 1000 - tt0284837
Item 27 / 1000 - tt2364897
Item 28 / 1000 - tt2235542
Item 29 / 1000 - tt0408985
Item 30 / 1000 - tt1185416
Item 31 / 1000 - tt1616195
Item 32 / 1000 - tt7214762
Item 33 / 1000 - tt4019560
Item 34 / 1000 - tt0455590
Item 35 / 1000 - tt3118874
Item 36 / 1000 - tt1290135
Item 37 / 1000 - tt6868216
Item 38 / 1000 - tt6214928
Item 39 /

Item 299 / 1000 - tt0401445
Item 300 / 1000 - tt3824344
Item 301 / 1000 - tt10147546
Item 302 / 1000 - tt2262315
Item 303 / 1000 - tt1319743
Item 304 / 1000 - tt0303151
Item 305 / 1000 - tt1655420
Item 306 / 1000 - tt7942742
Item 307 / 1000 - tt0433412
Item 308 / 1000 - tt4513674
Item 309 / 1000 - tt5461956
Item 310 / 1000 - tt0214641
Item 311 / 1000 - tt1067765
Item 312 / 1000 - tt0144528
Item 313 / 1000 - tt0832266
Item 314 / 1000 - tt5493706
Item 315 / 1000 - tt0349467
Item 316 / 1000 - tt6483364
Item 317 / 1000 - tt1244658
Item 318 / 1000 - tt3316960
Item 319 / 1000 - tt0398913
Item 320 / 1000 - tt2101473
Item 321 / 1000 - tt0443431
Item 322 / 1000 - tt2958484
Item 323 / 1000 - tt0770703
Item 324 / 1000 - tt2304953
Item 325 / 1000 - tt1836944
Item 326 / 1000 - tt1142800
Item 327 / 1000 - tt1189340
Item 328 / 1000 - tt0367631
Item 329 / 1000 - tt0472219
Item 330 / 1000 - tt0427327
Item 331 / 1000 - tt7755856
Item 332 / 1000 - tt0358135
Item 333 / 1000 - tt1523267
Item 334 / 1000 - t

Item 592 / 1000 - tt0259288
Item 593 / 1000 - tt0314166
Item 594 / 1000 - tt5251438
Item 595 / 1000 - tt1987028
Item 596 / 1000 - tt0215129
Item 597 / 1000 - tt0314412
Item 598 / 1000 - tt0817538
Item 599 / 1000 - tt0250468
Item 600 / 1000 - tt0461703
Item 601 / 1000 - tt2558318
Item 602 / 1000 - tt1821549
Item 603 / 1000 - tt1724970
Item 604 / 1000 - tt1227177
Item 605 / 1000 - tt1424432
Item 606 / 1000 - tt2229511
Item 607 / 1000 - tt1441953
Item 608 / 1000 - tt1178663
Item 609 / 1000 - tt1967545
Item 610 / 1000 - tt0243017
Item 611 / 1000 - tt0416716
Item 612 / 1000 - tt1206543
Item 613 / 1000 - tt0763831
Item 614 / 1000 - tt2645670
Item 615 / 1000 - tt1322306
Item 616 / 1000 - tt0416508
Item 617 / 1000 - tt0272152
Item 618 / 1000 - tt1758610
Item 619 / 1000 - tt2044801
Item 620 / 1000 - tt0875034
Item 621 / 1000 - tt0498399
Item 622 / 1000 - tt3038708
Item 623 / 1000 - tt2510028
Item 624 / 1000 - tt0903657
Item 625 / 1000 - tt0446755
Item 626 / 1000 - tt3520698
Item 627 / 1000 - tt

Item 885 / 1000 - tt4820224
Item 886 / 1000 - tt6574272
Item 887 / 1000 - tt0181196
Item 888 / 1000 - tt5805752
Item 889 / 1000 - tt1117581
Item 890 / 1000 - tt0134847
Item 891 / 1000 - tt2948202
Item 892 / 1000 - tt0844029
Item 893 / 1000 - tt4399952
Item 894 / 1000 - tt0455967
Item 895 / 1000 - tt6215208
Item 896 / 1000 - tt2309021
Item 897 / 1000 - tt2333804
Item 898 / 1000 - tt0367027
Item 899 / 1000 - tt2251281
Item 900 / 1000 - tt0438315
Item 901 / 1000 - tt4649416
Item 902 / 1000 - tt8743064
Item 903 / 1000 - tt2511428
Item 904 / 1000 - tt1331307
Item 905 / 1000 - tt5195412
Item 906 / 1000 - tt1372686
Item 907 / 1000 - tt1266029
Item 908 / 1000 - tt0498380
Item 909 / 1000 - tt10293884
Item 910 / 1000 - tt0476964
Item 911 / 1000 - tt5164184
Item 912 / 1000 - tt4769836
Item 913 / 1000 - tt2011109
Item 914 / 1000 - tt1047011
Item 915 / 1000 - tt0247745
Item 916 / 1000 - tt3859310
Item 917 / 1000 - tt0372532
Item 918 / 1000 - tt0383534
Item 919 / 1000 - tt0881891
Item 920 / 1000 - t

In [41]:
df5 = webscraper(df5)

Item 1 / 1000 - tt1416801


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt1391034
Item 3 / 1000 - tt3188560
Item 4 / 1000 - tt0395169
Item 5 / 1000 - tt0244970
Item 6 / 1000 - tt0264616
Item 7 / 1000 - tt0896534
Item 8 / 1000 - tt0219854
Item 9 / 1000 - tt1571243
Item 10 / 1000 - tt0325537
Item 11 / 1000 - tt0245120
Item 12 / 1000 - tt1007029
Item 13 / 1000 - tt1817081
Item 14 / 1000 - tt4758646
Item 15 / 1000 - tt0434215
Item 16 / 1000 - tt1114698
Item 17 / 1000 - tt0372237
Item 18 / 1000 - tt1833844
Item 19 / 1000 - tt0301526
Item 20 / 1000 - tt3488710
Item 21 / 1000 - tt1104835
Item 22 / 1000 - tt2101441
Item 23 / 1000 - tt4348012
Item 24 / 1000 - tt0279730
Item 25 / 1000 - tt4034354
Item 26 / 1000 - tt1790886
Item 27 / 1000 - tt0264472
Item 28 / 1000 - tt1313092
Item 29 / 1000 - tt2802136
Item 30 / 1000 - tt0443295
Item 31 / 1000 - tt1487931
Item 32 / 1000 - tt2145829
Item 33 / 1000 - tt2450186
Item 34 / 1000 - tt0838283
Item 35 / 1000 - tt7125860
Item 36 / 1000 - tt1104733
Item 37 / 1000 - tt0866437
Item 38 / 1000 - tt3672742
Item 39 /

Item 299 / 1000 - tt3438640
Item 300 / 1000 - tt1653665
Item 301 / 1000 - tt0368008
Item 302 / 1000 - tt0445922
Item 303 / 1000 - tt4129870
Item 304 / 1000 - tt0893382
Item 305 / 1000 - tt3864024
Item 306 / 1000 - tt0337697
Item 307 / 1000 - tt0430922
Item 308 / 1000 - tt0337909
Item 309 / 1000 - tt1412386
Item 310 / 1000 - tt1532503
Item 311 / 1000 - tt0940656
Item 312 / 1000 - tt0164212
Item 313 / 1000 - tt5460880
Item 314 / 1000 - tt8884364
Item 315 / 1000 - tt0211933
Item 316 / 1000 - tt0486578
Item 317 / 1000 - tt8488126
Item 318 / 1000 - tt6204340
Item 319 / 1000 - tt1535970
Item 320 / 1000 - tt7903530
Item 321 / 1000 - tt0303714
Item 322 / 1000 - tt1277737
Item 323 / 1000 - tt0376108
Item 324 / 1000 - tt3692652
Item 325 / 1000 - tt0803057
Item 326 / 1000 - tt5629964
Item 327 / 1000 - tt2404425
Item 328 / 1000 - tt3471098
Item 329 / 1000 - tt8998472
Item 330 / 1000 - tt1521197
Item 331 / 1000 - tt0899106
Item 332 / 1000 - tt0970411
Item 333 / 1000 - tt2577172
Item 334 / 1000 - tt

Item 590 / 1000 - tt1049405
Item 591 / 1000 - tt3922798
Item 592 / 1000 - tt1701990
Item 593 / 1000 - tt0288415
Item 594 / 1000 - tt2014400
Item 595 / 1000 - tt1702443
Item 596 / 1000 - tt1014801
Item 597 / 1000 - tt0246989
Item 598 / 1000 - tt0313542
Item 599 / 1000 - tt4978710
Item 600 / 1000 - tt2717860
Item 601 / 1000 - tt0458413
Item 602 / 1000 - tt2396589
Item 603 / 1000 - tt3138376
Item 604 / 1000 - tt0338427
Item 605 / 1000 - tt9326056
Item 606 / 1000 - tt2309961
Item 607 / 1000 - tt4411618
Item 608 / 1000 - tt0404802
Item 609 / 1000 - tt2011311
Item 610 / 1000 - tt4686844
Item 611 / 1000 - tt2167202
Item 612 / 1000 - tt1094198
Item 613 / 1000 - tt1700845
Item 614 / 1000 - tt1178665
Item 615 / 1000 - tt2235902
Item 616 / 1000 - tt0318974
Item 617 / 1000 - tt9577852
Item 618 / 1000 - tt6156350
Item 619 / 1000 - tt1493842
Item 620 / 1000 - tt0414055
Item 621 / 1000 - tt2380331
Item 622 / 1000 - tt0266489
Item 623 / 1000 - tt0436697
Item 624 / 1000 - tt2064968
Item 625 / 1000 - tt

Item 883 / 1000 - tt2281159
Item 884 / 1000 - tt5460858
Item 885 / 1000 - tt0279113
Item 886 / 1000 - tt1648204
Item 887 / 1000 - tt7369722
Item 888 / 1000 - tt2140373
Item 889 / 1000 - tt0466856
Item 890 / 1000 - tt1535612
Item 891 / 1000 - tt1788391
Item 892 / 1000 - tt2954474
Item 893 / 1000 - tt4686108
Item 894 / 1000 - tt1286130
Item 895 / 1000 - tt0268126
Item 896 / 1000 - tt3774790
Item 897 / 1000 - tt1581835
Item 898 / 1000 - tt0159784
Item 899 / 1000 - tt1196956
Item 900 / 1000 - tt0355702
Item 901 / 1000 - tt0246464
Item 902 / 1000 - tt3525168
Item 903 / 1000 - tt5281700
Item 904 / 1000 - tt0780607
Item 905 / 1000 - tt0397044
Item 906 / 1000 - tt2461520
Item 907 / 1000 - tt0261983
Item 908 / 1000 - tt2124787
Item 909 / 1000 - tt2452386
Item 910 / 1000 - tt1381418
Item 911 / 1000 - tt0353324
Item 912 / 1000 - tt0339412
Item 913 / 1000 - tt0323250
Item 914 / 1000 - tt5351000
Item 915 / 1000 - tt3686998
Item 916 / 1000 - tt0304415
Item 917 / 1000 - tt4917224
Item 918 / 1000 - tt

In [46]:
df6 = webscraper(df6)

Item 1 / 1000 - tt2094018


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt4501454
Item 3 / 1000 - tt3405276
Item 4 / 1000 - tt9165664
Item 5 / 1000 - tt4680980
Item 6 / 1000 - tt2224455
Item 7 / 1000 - tt8228172
Item 8 / 1000 - tt1212454
Item 9 / 1000 - tt0289408
Item 10 / 1000 - tt0443496
Item 11 / 1000 - tt2980210
Item 12 / 1000 - tt1959332
Item 13 / 1000 - tt0401815
Item 14 / 1000 - tt0838221
Item 15 / 1000 - tt1598822
Item 16 / 1000 - tt4368496
Item 17 / 1000 - tt1194417
Item 18 / 1000 - tt7208564
Item 19 / 1000 - tt0261289
Item 20 / 1000 - tt2345112
Item 21 / 1000 - tt0466839
Item 22 / 1000 - tt5653444
Item 23 / 1000 - tt0285742
Item 24 / 1000 - tt1995390
Item 25 / 1000 - tt0246677
Item 26 / 1000 - tt7073710
Item 27 / 1000 - tt0455782
Item 28 / 1000 - tt1194263
Item 29 / 1000 - tt0496634
Item 30 / 1000 - tt5591666
Item 31 / 1000 - tt0236348
Item 32 / 1000 - tt4827922
Item 33 / 1000 - tt1146438
Item 34 / 1000 - tt2292959
Item 35 / 1000 - tt0134630
Item 36 / 1000 - tt0278736
Item 37 / 1000 - tt3612320
Item 38 / 1000 - tt0365485
Item 39 /

Item 296 / 1000 - tt1814621
Item 297 / 1000 - tt0204626
Item 298 / 1000 - tt6881910
Item 299 / 1000 - tt1810697
Item 300 / 1000 - tt0324127
Item 301 / 1000 - tt1226766
Item 302 / 1000 - tt0210075
Item 303 / 1000 - tt0280969
Item 304 / 1000 - tt3302706
Item 305 / 1000 - tt2369317
Item 306 / 1000 - tt1743993
Item 307 / 1000 - tt0318850
Item 308 / 1000 - tt2366308
Item 309 / 1000 - tt0311648
Item 310 / 1000 - tt3300572
Item 311 / 1000 - tt1827579
Item 312 / 1000 - tt0235737
Item 313 / 1000 - tt0389957
Item 314 / 1000 - tt1073510
Item 315 / 1000 - tt1785635
Item 316 / 1000 - tt2392830
Item 317 / 1000 - tt1155056
Item 318 / 1000 - tt4814760
Item 319 / 1000 - tt4434688
Item 320 / 1000 - tt0266452
Item 321 / 1000 - tt6560164
Item 322 / 1000 - tt3233418
Item 323 / 1000 - tt0279111
Item 324 / 1000 - tt1314177
Item 325 / 1000 - tt2955096
Item 326 / 1000 - tt5540992
Item 327 / 1000 - tt3127022
Item 328 / 1000 - tt0292610
Item 329 / 1000 - tt6413868
Item 330 / 1000 - tt7440432
Item 331 / 1000 - tt

Item 587 / 1000 - tt0845046
Item 588 / 1000 - tt1389762
Item 589 / 1000 - tt0423169
Item 590 / 1000 - tt3026488
Item 591 / 1000 - tt0408524
Item 592 / 1000 - tt1478325
Item 593 / 1000 - tt0276919
Item 594 / 1000 - tt2317225
Item 595 / 1000 - tt3282076
Item 596 / 1000 - tt2396566
Item 597 / 1000 - tt2236182
Item 598 / 1000 - tt4991512
Item 599 / 1000 - tt4185572
Item 600 / 1000 - tt1498569
Item 601 / 1000 - tt0954947
Item 602 / 1000 - tt8329290
Item 603 / 1000 - tt0318374
Item 604 / 1000 - tt0445939
Item 605 / 1000 - tt1601475
Item 606 / 1000 - tt0283084
Item 607 / 1000 - tt8707374
Item 608 / 1000 - tt0470765
Item 609 / 1000 - tt5302658
Item 610 / 1000 - tt0189998
Item 611 / 1000 - tt2210463
Item 612 / 1000 - tt4638148
Item 613 / 1000 - tt3182620
Item 614 / 1000 - tt1610996
Item 615 / 1000 - tt1446147
Item 616 / 1000 - tt1183923
Item 617 / 1000 - tt5223342
Item 618 / 1000 - tt1183919
Item 619 / 1000 - tt5952332
Item 620 / 1000 - tt2885364
Item 621 / 1000 - tt0763840
Item 622 / 1000 - tt

Item 880 / 1000 - tt0800095
Item 881 / 1000 - tt1934458
Item 882 / 1000 - tt6346162
Item 883 / 1000 - tt0488658
Item 884 / 1000 - tt0985058
Item 885 / 1000 - tt6375308
Item 886 / 1000 - tt1895299
Item 887 / 1000 - tt1321860
Item 888 / 1000 - tt0387412
Item 889 / 1000 - tt2866708
Item 890 / 1000 - tt0213790
Item 891 / 1000 - tt1510906
Item 892 / 1000 - tt3544014
Item 893 / 1000 - tt3177316
Item 894 / 1000 - tt0274659
Item 895 / 1000 - tt0831887
Item 896 / 1000 - tt1286537
Item 897 / 1000 - tt0453494
Item 898 / 1000 - tt0369281
Item 899 / 1000 - tt4843046
Item 900 / 1000 - tt2725962
Item 901 / 1000 - tt1876451
Item 902 / 1000 - tt4701546
Item 903 / 1000 - tt1659216
Item 904 / 1000 - tt0480239
Item 905 / 1000 - tt2112210
Item 906 / 1000 - tt0457572
Item 907 / 1000 - tt0437232
Item 908 / 1000 - tt7440948
Item 909 / 1000 - tt0253798
Item 910 / 1000 - tt4975920
Item 911 / 1000 - tt2376218
Item 912 / 1000 - tt4200442
Item 913 / 1000 - tt1078885
Item 914 / 1000 - tt0323939
Item 915 / 1000 - tt

In [66]:
df7 = webscraper(df7)

Item 1 / 1000 - tt0365885


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt0424136
Item 3 / 1000 - tt0806027
Item 4 / 1000 - tt1603257
Item 5 / 1000 - tt0317042
Item 6 / 1000 - tt1795702
Item 7 / 1000 - tt2361184
Item 8 / 1000 - tt3491962
Item 9 / 1000 - tt2481480
Item 10 / 1000 - tt2395970
Item 11 / 1000 - tt2090629
Item 12 / 1000 - tt1999192
Item 13 / 1000 - tt1698648
Item 14 / 1000 - tt6081670
Item 15 / 1000 - tt1921070
Item 16 / 1000 - tt4661600
Item 17 / 1000 - tt1234654
Item 18 / 1000 - tt1880399
Item 19 / 1000 - tt0486358
Item 20 / 1000 - tt1959438
Item 21 / 1000 - tt0367188
Item 22 / 1000 - tt0282552
Item 23 / 1000 - tt2449638
Item 24 / 1000 - tt4146622
Item 25 / 1000 - tt5566790
Item 26 / 1000 - tt3478232
Item 27 / 1000 - tt0443456
Item 28 / 1000 - tt2781832
Item 29 / 1000 - tt0458438
Item 30 / 1000 - tt0854678
Item 31 / 1000 - tt0415978
Item 32 / 1000 - tt1606392
Item 33 / 1000 - tt0821638
Item 34 / 1000 - tt0184791
Item 35 / 1000 - tt2693580
Item 36 / 1000 - tt5153236
Item 37 / 1000 - tt0342172
Item 38 / 1000 - tt0435528
Item 39 /

Item 297 / 1000 - tt0988849
Item 298 / 1000 - tt0393685
Item 299 / 1000 - tt1422136
Item 300 / 1000 - tt1684226
Item 301 / 1000 - tt2414212
Item 302 / 1000 - tt2183034
Item 303 / 1000 - tt0186045
Item 304 / 1000 - tt0387059
Item 305 / 1000 - tt5352846
Item 306 / 1000 - tt2184287
Item 307 / 1000 - tt1781775
Item 308 / 1000 - tt0415965
Item 309 / 1000 - tt1097013
Item 310 / 1000 - tt0383694
Item 311 / 1000 - tt1229360
Item 312 / 1000 - tt1092633
Item 313 / 1000 - tt0361841
Item 314 / 1000 - tt1654523
Item 315 / 1000 - tt1135095
Item 316 / 1000 - tt1100053
Item 317 / 1000 - tt2977090
Item 318 / 1000 - tt3268340
Item 319 / 1000 - tt1147687
Item 320 / 1000 - tt5536736
Item 321 / 1000 - tt1781058
Item 322 / 1000 - tt0385705
Item 323 / 1000 - tt0804552
Item 324 / 1000 - tt7344360
Item 325 / 1000 - tt5340856
Item 326 / 1000 - tt0493424
Item 327 / 1000 - tt2034031
Item 328 / 1000 - tt3687398
Item 329 / 1000 - tt0787462
Item 330 / 1000 - tt1135933
Item 331 / 1000 - tt0323033
Item 332 / 1000 - tt

Item 590 / 1000 - tt0435665
Item 591 / 1000 - tt4547120
Item 592 / 1000 - tt1867093
Item 593 / 1000 - tt1185431
Item 594 / 1000 - tt2040560
Item 595 / 1000 - tt0258038
Item 596 / 1000 - tt6719524
Item 597 / 1000 - tt2885628
Item 598 / 1000 - tt0179098
Item 599 / 1000 - tt2027154
Item 600 / 1000 - tt0867334
Item 601 / 1000 - tt0439771
Item 602 / 1000 - tt1935156
Item 603 / 1000 - tt6263642
Item 604 / 1000 - tt3505712
Item 605 / 1000 - tt4381236
Item 606 / 1000 - tt3090670
Item 607 / 1000 - tt0800003
Item 608 / 1000 - tt0408196
Item 609 / 1000 - tt0122459
Item 610 / 1000 - tt1714209
Item 611 / 1000 - tt1814643
Item 612 / 1000 - tt0363780
Item 613 / 1000 - tt0454776
Item 614 / 1000 - tt1092634
Item 615 / 1000 - tt1064884
Item 616 / 1000 - tt6649108
Item 617 / 1000 - tt0472062
Item 618 / 1000 - tt0285627
Item 619 / 1000 - tt3910814
Item 620 / 1000 - tt1020543
Item 621 / 1000 - tt1139282
Item 622 / 1000 - tt0772178
Item 623 / 1000 - tt1640711
Item 624 / 1000 - tt0307535
Item 625 / 1000 - tt

Item 883 / 1000 - tt2028530
Item 884 / 1000 - tt0250934
Item 885 / 1000 - tt0818165
Item 886 / 1000 - tt5521550
Item 887 / 1000 - tt2980626
Item 888 / 1000 - tt1407065
Item 889 / 1000 - tt0905994
Item 890 / 1000 - tt0799954
Item 891 / 1000 - tt4872078
Item 892 / 1000 - tt2012665
Item 893 / 1000 - tt0948547
Item 894 / 1000 - tt2334733
Item 895 / 1000 - tt0251806
Item 896 / 1000 - tt2120152
Item 897 / 1000 - tt0491747
Item 898 / 1000 - tt1107859
Item 899 / 1000 - tt5710042
Item 900 / 1000 - tt0448172
Item 901 / 1000 - tt1697851
Item 902 / 1000 - tt1682181
Item 903 / 1000 - tt0314786
Item 904 / 1000 - tt3726704
Item 905 / 1000 - tt1579232
Item 906 / 1000 - tt0204137
Item 907 / 1000 - tt5751998
Item 908 / 1000 - tt1700844
Item 909 / 1000 - tt0271271
Item 910 / 1000 - tt3762912
Item 911 / 1000 - tt0372884
Item 912 / 1000 - tt7584702
Item 913 / 1000 - tt1957945
Item 914 / 1000 - tt3520290
Item 915 / 1000 - tt0413879
Item 916 / 1000 - tt6883152
Item 917 / 1000 - tt4145178
Item 918 / 1000 - tt

In [54]:
df8 = webscraper(df8)

Item 1 / 1000 - tt0892391


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt4728386
Item 3 / 1000 - tt2509850
Item 4 / 1000 - tt1683048
Item 5 / 1000 - tt3966544
Item 6 / 1000 - tt1440732
Item 7 / 1000 - tt1352388
Item 8 / 1000 - tt2089750
Item 9 / 1000 - tt0486640
Item 10 / 1000 - tt1174730
Item 11 / 1000 - tt0264761
Item 12 / 1000 - tt0367089
Item 13 / 1000 - tt0772157
Item 14 / 1000 - tt4607722
Item 15 / 1000 - tt3662066
Item 16 / 1000 - tt2040578
Item 17 / 1000 - tt1558250
Item 18 / 1000 - tt7588798
Item 19 / 1000 - tt3343868
Item 20 / 1000 - tt1082886
Item 21 / 1000 - tt2104837
Item 22 / 1000 - tt0298228
Item 23 / 1000 - tt4269118
Item 24 / 1000 - tt0479113
Item 25 / 1000 - tt5255524
Item 26 / 1000 - tt0377091
Item 27 / 1000 - tt0328802
Item 28 / 1000 - tt3138122
Item 29 / 1000 - tt2286990
Item 30 / 1000 - tt4305148
Item 31 / 1000 - tt4105970
Item 32 / 1000 - tt0974554
Item 33 / 1000 - tt1312203
Item 34 / 1000 - tt3327624
Item 35 / 1000 - tt2036376
Item 36 / 1000 - tt1680114
Item 37 / 1000 - tt3018070
Item 38 / 1000 - tt2191888
Item 39 /

Item 299 / 1000 - tt2107861
Item 300 / 1000 - tt3445270
Item 301 / 1000 - tt2556114
Item 302 / 1000 - tt0390393
Item 303 / 1000 - tt5990444
Item 304 / 1000 - tt4552524
Item 305 / 1000 - tt1738387
Item 306 / 1000 - tt5883570
Item 307 / 1000 - tt0258068
Item 308 / 1000 - tt1934452
Item 309 / 1000 - tt2403558
Item 310 / 1000 - tt3885524
Item 311 / 1000 - tt4335650
Item 312 / 1000 - tt4044464
Item 313 / 1000 - tt0372334
Item 314 / 1000 - tt3016748
Item 315 / 1000 - tt0473356
Item 316 / 1000 - tt3021360
Item 317 / 1000 - tt4354756
Item 318 / 1000 - tt1309379
Item 319 / 1000 - tt0278723
Item 320 / 1000 - tt0398027
Item 321 / 1000 - tt0758771
Item 322 / 1000 - tt1747960
Item 323 / 1000 - tt0418586
Item 324 / 1000 - tt8128188
Item 325 / 1000 - tt5997848
Item 326 / 1000 - tt0313285
Item 327 / 1000 - tt0379557
Item 328 / 1000 - tt0262396
Item 329 / 1000 - tt1733578
Item 330 / 1000 - tt0981224
Item 331 / 1000 - tt0475417
Item 332 / 1000 - tt0492486
Item 333 / 1000 - tt1471324
Item 334 / 1000 - tt

Item 592 / 1000 - tt3138558
Item 593 / 1000 - tt3272570
Item 594 / 1000 - tt3097204
Item 595 / 1000 - tt0420560
Item 596 / 1000 - tt0203119
Item 597 / 1000 - tt0841044
Item 598 / 1000 - tt2582426
Item 599 / 1000 - tt0286947
Item 600 / 1000 - tt0795439
Item 601 / 1000 - tt0315824
Item 602 / 1000 - tt0810823
Item 603 / 1000 - tt3463106
Item 604 / 1000 - tt0326900
Item 605 / 1000 - tt3676370
Item 606 / 1000 - tt0846308
Item 607 / 1000 - tt0357054
Item 608 / 1000 - tt5039088
Item 609 / 1000 - tt1130969
Item 610 / 1000 - tt5701718
Item 611 / 1000 - tt3317208
Item 612 / 1000 - tt3131456
Item 613 / 1000 - tt0896923
Item 614 / 1000 - tt6418778
Item 615 / 1000 - tt2927212
Item 616 / 1000 - tt4016226
Item 617 / 1000 - tt2551396
Item 618 / 1000 - tt0469021
Item 619 / 1000 - tt0366777
Item 620 / 1000 - tt1407078
Item 621 / 1000 - tt4145324
Item 622 / 1000 - tt0455326
Item 623 / 1000 - tt7468616
Item 624 / 1000 - tt0252444
Item 625 / 1000 - tt8110630
Item 626 / 1000 - tt1815852
Item 627 / 1000 - tt

Item 882 / 1000 - tt0357470
Item 883 / 1000 - tt1332128
Item 884 / 1000 - tt4661798
Item 885 / 1000 - tt1473397
Item 886 / 1000 - tt1430132
Item 887 / 1000 - tt1870529
Item 888 / 1000 - tt7370936
Item 889 / 1000 - tt7689958
Item 890 / 1000 - tt6333064
Item 891 / 1000 - tt2357788
Item 892 / 1000 - tt2139851
Item 893 / 1000 - tt5605328
Item 894 / 1000 - tt5728684
Item 895 / 1000 - tt1673697
Item 896 / 1000 - tt1514041
Item 897 / 1000 - tt3486392
Item 898 / 1000 - tt4661680
Item 899 / 1000 - tt0494222
Item 900 / 1000 - tt7328154
Item 901 / 1000 - tt1241017
Item 902 / 1000 - tt3746298
Item 903 / 1000 - tt1104806
Item 904 / 1000 - tt0427528
Item 905 / 1000 - tt1865543
Item 906 / 1000 - tt1621446
Item 907 / 1000 - tt4938484
Item 908 / 1000 - tt2494280
Item 909 / 1000 - tt1785612
Item 910 / 1000 - tt1562568
Item 911 / 1000 - tt3593046
Item 912 / 1000 - tt1286821
Item 913 / 1000 - tt0349416
Item 914 / 1000 - tt1784538
Item 915 / 1000 - tt2217895
Item 916 / 1000 - tt3820450
Item 917 / 1000 - tt

In [57]:
df9 = webscraper(df9)

Item 1 / 1000 - tt0443450


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt3328716
Item 3 / 1000 - tt0247164
Item 4 / 1000 - tt3333870
Item 5 / 1000 - tt1628055
Item 6 / 1000 - tt1552624
Item 7 / 1000 - tt1477855
Item 8 / 1000 - tt2718440
Item 9 / 1000 - tt0940580
Item 10 / 1000 - tt3687186
Item 11 / 1000 - tt1991031
Item 12 / 1000 - tt0156323
Item 13 / 1000 - tt2459156
Item 14 / 1000 - tt2909476
Item 15 / 1000 - tt0462160
Item 16 / 1000 - tt5113926
Item 17 / 1000 - tt4630444
Item 18 / 1000 - tt3194590
Item 19 / 1000 - tt2396224
Item 20 / 1000 - tt1972819
Item 21 / 1000 - tt1198408
Item 22 / 1000 - tt3323638
Item 23 / 1000 - tt3145220
Item 24 / 1000 - tt1312251
Item 25 / 1000 - tt2130142
Item 26 / 1000 - tt2245151
Item 27 / 1000 - tt2378884
Item 28 / 1000 - tt0254334
Item 29 / 1000 - tt4680568
Item 30 / 1000 - tt1447793
Item 31 / 1000 - tt0318763
Item 32 / 1000 - tt0390336
Item 33 / 1000 - tt0430576
Item 34 / 1000 - tt9056818
Item 35 / 1000 - tt1986994
Item 36 / 1000 - tt1563704
Item 37 / 1000 - tt0482461
Item 38 / 1000 - tt3004852
Item 39 /

Item 299 / 1000 - tt2948078
Item 300 / 1000 - tt1957938
Item 301 / 1000 - tt0266391
Item 302 / 1000 - tt0368711
Item 303 / 1000 - tt0289635
Item 304 / 1000 - tt9308456
Item 305 / 1000 - tt0343996
Item 306 / 1000 - tt1638364
Item 307 / 1000 - tt5692030
Item 308 / 1000 - tt2042447
Item 309 / 1000 - tt8106570
Item 310 / 1000 - tt3382842
Item 311 / 1000 - tt5862902
Item 312 / 1000 - tt3704416
Item 313 / 1000 - tt4172224
Item 314 / 1000 - tt0221069
Item 315 / 1000 - tt0430674
Item 316 / 1000 - tt5886510
Item 317 / 1000 - tt0398842
Item 318 / 1000 - tt6864864
Item 319 / 1000 - tt1176724
Item 320 / 1000 - tt0451102
Item 321 / 1000 - tt7825514
Item 322 / 1000 - tt0988083
Item 323 / 1000 - tt0449077
Item 324 / 1000 - tt1855401
Item 325 / 1000 - tt3311384
Item 326 / 1000 - tt2663812
Item 327 / 1000 - tt0271219
Item 328 / 1000 - tt0258273
Item 329 / 1000 - tt5158522
Item 330 / 1000 - tt8271714
Item 331 / 1000 - tt0396857
Item 332 / 1000 - tt0156841
Item 333 / 1000 - tt5456798
Item 334 / 1000 - tt

Item 591 / 1000 - tt0910847
Item 592 / 1000 - tt7410684
Item 593 / 1000 - tt1704292
Item 594 / 1000 - tt1942971
Item 595 / 1000 - tt0851430
Item 596 / 1000 - tt2125653
Item 597 / 1000 - tt1865573
Item 598 / 1000 - tt0239986
Item 599 / 1000 - tt5434972
Item 600 / 1000 - tt1517249
Item 601 / 1000 - tt2689958
Item 602 / 1000 - tt1555126
Item 603 / 1000 - tt1485749
Item 604 / 1000 - tt4711924
Item 605 / 1000 - tt0206367
Item 606 / 1000 - tt0204640
Item 607 / 1000 - tt1865393
Item 608 / 1000 - tt1135924
Item 609 / 1000 - tt1020876
Item 610 / 1000 - tt0366177
Item 611 / 1000 - tt2708254
Item 612 / 1000 - tt6915100
Item 613 / 1000 - tt3205394
Item 614 / 1000 - tt5070196
Item 615 / 1000 - tt0218378
Item 616 / 1000 - tt5700176
Item 617 / 1000 - tt1596753
Item 618 / 1000 - tt10009796
Item 619 / 1000 - tt7545566
Item 620 / 1000 - tt1541123
Item 621 / 1000 - tt6688354
Item 622 / 1000 - tt0944101
Item 623 / 1000 - tt5097070
Item 624 / 1000 - tt1087527
Item 625 / 1000 - tt2475846
Item 626 / 1000 - t

Item 883 / 1000 - tt0360323
Item 884 / 1000 - tt0206917
Item 885 / 1000 - tt3157224
Item 886 / 1000 - tt0182295
Item 887 / 1000 - tt0339034
Item 888 / 1000 - tt3294200
Item 889 / 1000 - tt5278578
Item 890 / 1000 - tt5115546
Item 891 / 1000 - tt1667418
Item 892 / 1000 - tt4441422
Item 893 / 1000 - tt0430919
Item 894 / 1000 - tt3564200
Item 895 / 1000 - tt1533089
Item 896 / 1000 - tt2133239
Item 897 / 1000 - tt1103984
Item 898 / 1000 - tt1368440
Item 899 / 1000 - tt7374952
Item 900 / 1000 - tt3082826
Item 901 / 1000 - tt1785669
Item 902 / 1000 - tt2113659
Item 903 / 1000 - tt3110960
Item 904 / 1000 - tt0381442
Item 905 / 1000 - tt1935065
Item 906 / 1000 - tt5537228
Item 907 / 1000 - tt1772373
Item 908 / 1000 - tt4712076
Item 909 / 1000 - tt7942746
Item 910 / 1000 - tt1844203
Item 911 / 1000 - tt3703148
Item 912 / 1000 - tt3108244
Item 913 / 1000 - tt7647010
Item 914 / 1000 - tt2362991
Item 915 / 1000 - tt3379814
Item 916 / 1000 - tt0421045
Item 917 / 1000 - tt4016250
Item 918 / 1000 - tt

In [62]:
df10 = webscraper(df10)

Item 1 / 1000 - tt0472582


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Item 2 / 1000 - tt4669252
Item 3 / 1000 - tt1989485
Item 4 / 1000 - tt1772261
Item 5 / 1000 - tt8201924
Item 6 / 1000 - tt2210834
Item 7 / 1000 - tt1270296
Item 8 / 1000 - tt2290709
Item 9 / 1000 - tt3083008
Item 10 / 1000 - tt2709784
Item 11 / 1000 - tt0770829
Item 12 / 1000 - tt0472156
Item 13 / 1000 - tt1730687
Item 14 / 1000 - tt2821314
Item 15 / 1000 - tt7264080
Item 16 / 1000 - tt1468829
Item 17 / 1000 - tt2450440
Item 18 / 1000 - tt0462219
Item 19 / 1000 - tt0892047
Item 20 / 1000 - tt1727261
Item 21 / 1000 - tt0848552
Item 22 / 1000 - tt4592572
Item 23 / 1000 - tt0194368
Item 24 / 1000 - tt2294729
Item 25 / 1000 - tt0804550
Item 26 / 1000 - tt1839482
Item 27 / 1000 - tt1988781
Item 28 / 1000 - tt0487037
Item 29 / 1000 - tt0388285
Item 30 / 1000 - tt1770734
Item 31 / 1000 - tt2572632
Item 32 / 1000 - tt0296845
Item 33 / 1000 - tt9652782
Item 34 / 1000 - tt0382937
Item 35 / 1000 - tt6969946
Item 36 / 1000 - tt1384927
Item 37 / 1000 - tt0120467
Item 38 / 1000 - tt4293752
Item 39 /

Item 299 / 1000 - tt2836376
Item 300 / 1000 - tt1300155
Item 301 / 1000 - tt1699147
Item 302 / 1000 - tt3120054
Item 303 / 1000 - tt2369041
Item 304 / 1000 - tt1837574
Item 305 / 1000 - tt1043844
Item 306 / 1000 - tt0985025
Item 307 / 1000 - tt0246592
Item 308 / 1000 - tt5672286
Item 309 / 1000 - tt1718158
Item 310 / 1000 - tt4765240
Item 311 / 1000 - tt0435687
Item 312 / 1000 - tt1235170
Item 313 / 1000 - tt1313244
Item 314 / 1000 - tt0350934
Item 315 / 1000 - tt2387589
Item 316 / 1000 - tt4530184
Item 317 / 1000 - tt3567194
Item 318 / 1000 - tt4340888
Item 319 / 1000 - tt1426363
Item 320 / 1000 - tt0395585
Item 321 / 1000 - tt0409799
Item 322 / 1000 - tt5095510
Item 323 / 1000 - tt1024744
Item 324 / 1000 - tt6349282
Item 325 / 1000 - tt3922816
Item 326 / 1000 - tt0388230
Item 327 / 1000 - tt0265307
Item 328 / 1000 - tt5210048
Item 329 / 1000 - tt0816545
Item 330 / 1000 - tt0283043
Item 331 / 1000 - tt4122118
Item 332 / 1000 - tt1097239
Item 333 / 1000 - tt4864624
Item 334 / 1000 - tt

Item 591 / 1000 - tt1029134
Item 592 / 1000 - tt0443455
Item 593 / 1000 - tt6333072
Item 594 / 1000 - tt2315500
Item 595 / 1000 - tt4743562
Item 596 / 1000 - tt4714568
Item 597 / 1000 - tt1190858
Item 598 / 1000 - tt1276105
Item 599 / 1000 - tt0412637
Item 600 / 1000 - tt4991652
Item 601 / 1000 - tt0385056
Item 602 / 1000 - tt2112293
Item 603 / 1000 - tt3699372
Item 604 / 1000 - tt7012864
Item 605 / 1000 - tt1772271
Item 606 / 1000 - tt4072326
Item 607 / 1000 - tt3731196
Item 608 / 1000 - tt2948160
Item 609 / 1000 - tt1251725
Item 610 / 1000 - tt4065324
Item 611 / 1000 - tt2352488
Item 612 / 1000 - tt0212985
Item 613 / 1000 - tt3517044
Item 614 / 1000 - tt1365483
Item 615 / 1000 - tt1489167
Item 616 / 1000 - tt3576038
Item 617 / 1000 - tt1474311
Item 618 / 1000 - tt1733732
Item 619 / 1000 - tt0912593
Item 620 / 1000 - tt6164014
Item 621 / 1000 - tt2484120
Item 622 / 1000 - tt3262990
Item 623 / 1000 - tt0487195
Item 624 / 1000 - tt5594080
Item 625 / 1000 - tt1318522
Item 626 / 1000 - tt

Item 882 / 1000 - tt1051713
Item 883 / 1000 - tt6861784
Item 884 / 1000 - tt2274356
Item 885 / 1000 - tt2385074
Item 886 / 1000 - tt1911607
Item 887 / 1000 - tt2204379
Item 888 / 1000 - tt1085507
Item 889 / 1000 - tt3219194
Item 890 / 1000 - tt1520496
Item 891 / 1000 - tt1003034
Item 892 / 1000 - tt0492330
Item 893 / 1000 - tt1623757
Item 894 / 1000 - tt2537390
Item 895 / 1000 - tt1048174
Item 896 / 1000 - tt0403407
Item 897 / 1000 - tt1077248
Item 898 / 1000 - tt0462295
Item 899 / 1000 - tt1400515
Item 900 / 1000 - tt0381111
Item 901 / 1000 - tt2182019
Item 902 / 1000 - tt0383393
Item 903 / 1000 - tt2152198
Item 904 / 1000 - tt4197642
Item 905 / 1000 - tt2625598
Item 906 / 1000 - tt2229842
Item 907 / 1000 - tt3283864
Item 908 / 1000 - tt1561768
Item 909 / 1000 - tt6813264
Item 910 / 1000 - tt7006938
Item 911 / 1000 - tt0480025
Item 912 / 1000 - tt0446750
Item 913 / 1000 - tt2059244
Item 914 / 1000 - tt2058636
Item 915 / 1000 - tt1442576
Item 916 / 1000 - tt4106306
Item 917 / 1000 - tt

In [67]:
#attach the 5 dataframes together
tmdb_bom = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10])

In [4]:
tmdb_bom

,id,studio,rating,budget,dom_gross,world_gross
0,tt2380307,Walt Disney Studios Motion Pictures,PG,NaN,210460015.0,8.078179e+08
1,tt7286456,Warner Bros.,R,55000000.0,335451311.0,1.074251e+09
2,tt7349950,Warner Bros.,R,79000000.0,211593228.0,4.730932e+08
3,tt4520988,Walt Disney Studios Motion Pictures,PG,150000000.0,477373578.0,1.450027e+09
4,tt7975244,Sony Pictures Entertainment (SPE),PG-13,125000000.0,320314960.0,8.000597e+08
...,...,...,...,...,...,...
9995,tt0349080,NaN,0,NaN,NaN,NaN
9996,tt0366178,NaN,0,NaN,NaN,NaN
9997,tt2501618,NaN,0,NaN,NaN,NaN
9998,tt0340376,Sony Pictures Classics,R,27000000.0,765637.0,1.079822e+06


In [8]:
# Setting the title as the index
tmdb_bom.set_index('id', inplace=True)

# Data Export

In [9]:
#exporting the dataframe to a csv
tmdb_bom.to_csv('api_data/tmdb_bom_scraped.csv')

# TO DO

Write Docstrings for all functions